# Translator
* https://docs.microsoft.com/ja-jp/azure/cognitive-services/translator/reference/v3-0-reference

In [1]:
# Cognitive Servicesの翻訳のリソースを作成する

In [2]:
base_url = 'api.cognitive.microsofttranslator.com'
key = 'xxxx'

In [3]:
import urllib, urllib.parse, urllib.error, urllib.request, json, http.client

# RegionもAzure PortalのKeyとEndpointから見れる
headers = {
    'Content-Type':'application/json',
    'Ocp-Apim-Subscription-Key':key,
    'Ocp-Apim-Subscription-Region':'japaneast'
}

body = [
  {
      'Text':'Hola mundo, ¿cómo están?'
  },
  {
      'Text':'Bon jour, mon amie!'
  },
  {
      'Text':'你好吗?'
  }
]

params = urllib.parse.urlencode({
    'api-version':'3.0',
    'to':'de'
})

print(params)

api-version=3.0&to=de


In [4]:
# UTF-8 にエンコード
str(body).encode('UTF-8')

b"[{'Text': 'Hola mundo, \xc2\xbfc\xc3\xb3mo est\xc3\xa1n?'}, {'Text': 'Bon jour, mon amie!'}, {'Text': '\xe4\xbd\xa0\xe5\xa5\xbd\xe5\x90\x97?'}]"

In [5]:
try:
    conn = http.client.HTTPSConnection(base_url)
    # paramsでは同じkeyに対して複数のパラメータを指定できないので、英語にもしたければ下のrequestに直接打つと良い
    conn.request('POST', '/translate?%s&to=en' % params, str(body).encode('UTF-8'), headers)
    response = conn.getresponse()
    jsonData = response.read().decode('UTF-8')
    data = json.loads(jsonData)
    print(data)
    conn.close()
except Exception as ex:
    print(ex)

[{'detectedLanguage': {'language': 'es', 'score': 1.0}, 'translations': [{'text': 'Hallo Welt, wie geht es dir?', 'to': 'de'}, {'text': 'Hello world, how are you?', 'to': 'en'}]}, {'detectedLanguage': {'language': 'fr', 'score': 1.0}, 'translations': [{'text': 'Guten Tag, mein Freund!', 'to': 'de'}, {'text': 'Good day, my friend!', 'to': 'en'}]}, {'detectedLanguage': {'language': 'zh-Hans', 'score': 1.0}, 'translations': [{'text': 'Wie geht es dir?', 'to': 'de'}, {'text': 'How are you?', 'to': 'en'}]}]


# Identify Languages
* https://westus2.dev.cognitive.microsoft.com/docs/services/TextAnalytics-v2-1/operations/56f30ceeeda5650db055a3c7

In [6]:
# Cognitive Servicesの言語サービスのリソースを作成する

In [7]:
ta_key = 'xxxx'
ta_base_url = 'lpalanguage0701.cognitiveservices.azure.com'

In [8]:
ta_headers = {
    'Content-Type':'application/json',
    'Ocp-Apim-Subscription-Key':ta_key,
}

ta_body = {
    'documents':[
      {
          'id':'1',
          'text':'Hola a todos'
      }         
    ]
}

ta_params = urllib.parse.urlencode({})

In [9]:
iso_name = 'en' # デフォルトを英語に設定
try:
    conn = http.client.HTTPSConnection(ta_base_url)
    conn.request('POST', '/text/analytics/v2.1/languages?%s' % ta_params, str(ta_body), ta_headers)
    response = conn.getresponse()
    jsonData = response.read().decode('UTF-8')
    data = json.loads(jsonData)
    print('data: ',data)
    for document in data['documents']:
        print('Document ' + document['id'])
        for lan in document['detectedLanguages']:
            iso_name = lan['iso6391Name']
            print(' ' + lan['iso6391Name'])
    conn.close()
except Exception as ex:
    print(ex)

data:  {'documents': [{'id': '1', 'detectedLanguages': [{'name': 'Spanish', 'iso6391Name': 'es', 'score': 1.0}]}], 'errors': []}
Document 1
 es


In [10]:
# このように書いて、Translatorにつなげることでどの言語からの翻訳かを指定可能
params = urllib.parse.urlencode({
    'api-version':'3.0',
    'to':'de',
    'from':iso_name
})

# Speech Translate
* https://docs.microsoft.com/ja-jp/azure/cognitive-services/speech-service/get-started-speech-translation?tabs=terminal&pivots=programming-language-python

In [11]:
# Cognitive ServicesのSpeechリソースを作成する

In [12]:
!pip install azure-cognitiveservices-speech

In [13]:
import os
import azure.cognitiveservices.speech as speechsdk

In [14]:
speech_key = 'xxxx'
speech_region = 'eastus'

In [15]:
from_language = 'en-US'
to_language = ['ja', 'zh-Hans'] # zh-Hans:Chinese

In [18]:
config = speechsdk.translation.SpeechTranslationConfig(subscription=speech_key, region=speech_region)
config.speech_recognition_language = from_language
for lan in to_language:
    config.add_target_language(lan)

recognizer = speechsdk.translation.TranslationRecognizer(translation_config=config)

print('Start talking....')
result = recognizer.recognize_once()
print(result)

Start talking....
TranslationRecognitionResult(result_id=5b7965cfdf0a4387b1dd548a54543a89, translations={'ja': 'こんにちは、はじめまして。', 'zh-Hans': '嗨，很高兴见到你。'}, reason=ResultReason.TranslatedSpeech)


In [19]:
print(result.text + ' translated to:')
for lan in to_language:
    print(lan + ': ' + result.translations[lan])

Hi, nice to meet you. translated to:
ja: こんにちは、はじめまして。
zh-Hans: 嗨，很高兴见到你。
